In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data

ModuleNotFoundError: No module named 'torch'

In [2]:
%pip install torch

  Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached triton-3.0.0-1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl (797.1 MB)
Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.w

In [ ]:
# Création d'un exemple de graphe
# Nœuds (stations)
num_nodes = 10
features = torch.randn((num_nodes, 5))  # 5 caractéristiques par nœud

# Arêtes (connectivité entre stations)
edges = torch.tensor([[0, 1, 1, 2, 3, 4, 4, 5],
                      [1, 0, 2, 1, 4, 3, 5, 4]], dtype=torch.long)  # Edges

# Poids des arêtes (temps de trajet ou nombre de passagers)
edge_weights = torch.tensor([10, 10, 15, 15, 5, 5, 20, 20], dtype=torch.float)

# Création du graphe PyG
data = Data(x=features, edge_index=edges, edge_attr=edge_weights)

In [ ]:
# Implémentation du GAT
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=2):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Instanciation du modèle
model = GAT(input_dim=5, hidden_dim=16, output_dim=3)
out = model(data)
print(out)


In [ ]:
# Implémentation du GCN
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Instanciation du modèle
model = GCN(input_dim=5, hidden_dim=16, output_dim=3)  # Exemple avec 3 classes
out = model(data)
print(out)

In [ ]:
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()


In [ ]:
def train_model(model, data, epochs=200, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        loss = train(model, data, optimizer)
        if epoch % 10 == 0:
            print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')
